链允许我们将多个组件组合在一起，创建一个单一的、一致的应用程序。例如，我们可以创建一个链，该链接接受用户输入，使用 PromptTemplate 对其进行格式化，然后将格式化后的响应传递给 LLM。我们可以通过将多个链组合在一起，或者通过将链与其他组件组合在一起，来构建更复杂的链。

In [1]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)


现在我们可以创建一个非常简单的链，它将接受用户输入，用它格式化提示符，然后将其发送到 LLM。

In [2]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("colorful socks"))




Sock It To Me!


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="What is a good name for a company that makes {product}?",
            input_variables=["product"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.9)
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
print(chain.run("colorful socks"))


Socktastic


In [4]:
chat = ChatOpenAI(temperature=0)
prompt_template = "Tell me a {adjective} joke"
llm_chain = LLMChain(
    llm=chat,
    prompt=PromptTemplate.from_template(prompt_template)
)

llm_chain(inputs={"adjective":"corny"})


{'adjective': 'corny',
 'text': "Sure, here's a corny joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!"}

In [5]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

conversation.run("Answer briefly. What are the first 3 colors of a rainbow?")
# -> The first three colors of a rainbow are red, orange, and yellow.
conversation.run("And the next 4?")
# -> The next four colors of a rainbow are green, blue, indigo, and violet.


'The next four colors of a rainbow are green, blue, indigo, and violet.'

 LLM Chain  多个输入，多个输出

In [6]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "What is a good name for a company that makes {product}?"

llm = OpenAI(temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain("colorful socks")


{'product': 'colorful socks', 'text': '\n\nSocktastic!'}

In [7]:
input_list = [
    {"product": "socks"},
    {"product": "computer"},
    {"product": "shoes"}
]

llm_chain.apply(input_list)

[{'text': '\n\nCozy Toes Socks.'},
 {'text': '\n\nTechCore Solutions.'},
 {'text': '\n\nFootwear Factory.'}]

In [8]:
llm_chain.generate(input_list)


LLMResult(generations=[[Generation(text='\n\nSocktastic!', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nTechCore Solutions.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nFootwear Factory.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'completion_tokens': 19, 'total_tokens': 55, 'prompt_tokens': 36}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('e901ea99-ba97-423c-9a02-5323fb3fec92')), RunInfo(run_id=UUID('25b3d644-6a20-4f70-9866-28bf15e51eb5')), RunInfo(run_id=UUID('e0b123d4-bd3d-4bab-a162-d78afa79ef54'))])

In [9]:
llm_chain.predict(product="colorful socks")


'\n\nSocktastic!'

In [10]:
# Multiple inputs example

template = """Tell me a {adjective} joke about {subject}."""
prompt = PromptTemplate(template=template, input_variables=["adjective", "subject"])
llm_chain = LLMChain(prompt=prompt, llm=OpenAI(temperature=0))

llm_chain.predict(adjective="sad", subject="ducks")


'\n\nQ: What did the duck say when his friend died?\nA: Quack, quack, goodbye.'

In [11]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all the colors in a rainbow"""
prompt = PromptTemplate(template=template, input_variables=[], output_parser=output_parser)
llm_chain = LLMChain(prompt=prompt, llm=llm)

llm_chain.predict()


'\n\nRed, orange, yellow, green, blue, indigo, violet'

In [12]:
llm_chain.predict_and_parse()


D:\code\python-gpt-course-main\venv\lib\site-packages\langchain\chains\llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


['Red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet']

SimpleSequentialChain：最简单的顺序链形式，每个步骤都有一个单一的输入/输出，一个步骤的输出是下一个步骤的输入。


In [13]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
 # This is an LLMChain to write a synopsis given a title of a play.
llm = OpenAI(temperature=.7)
template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [14]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=.7)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [15]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)


In [16]:
review = overall_chain.run("Tragedy at sunset on the beach")




> Entering new SimpleSequentialChain chain...


Tragedy at Sunset on the Beach follows the story of a young couple, Jack and Emma, who have been happily in love for years. They decide to take a romantic trip to a beachside resort for their anniversary. As they enjoy the beautiful sunset on the beach, tragedy strikes. Jack is suddenly struck by a mysterious illness and must be rushed to the hospital. Emma is left alone to cope with the sudden change in circumstances and is forced to reconsider the meaning of love and life. As the play progresses, Emma discovers the truth behind Jack's illness and must make the difficult decision of whether to stay or to leave. This story of love, loss, and reconciliation is sure to leave an impression on audiences.


Tragedy at Sunset on the Beach is a powerful and moving story of love, loss, and reconciliation. The play follows the story of Jack and Emma, a young couple deeply in love and excited to celebrate their anniversary at a beachside resort. 

In [17]:
print(review)




Tragedy at Sunset on the Beach is a powerful and moving story of love, loss, and reconciliation. The play follows the story of Jack and Emma, a young couple deeply in love and excited to celebrate their anniversary at a beachside resort. However, tragedy strikes when Jack is struck by a mysterious illness and is rushed to the hospital. Emma is left alone to cope with the sudden change in circumstances and must reconsider the meaning of love and life. The play does an excellent job of exploring the various emotions and complexities of love and loss and how it can affect someone's life. The performances by the actors are nuanced and heartfelt, making the characters' struggles and triumphs all the more powerful and moving. Tragedy at Sunset on the Beach is sure to leave an impression on audiences and is not to be missed.


SequentialChain：更一般的顺序链形式，允许多个输入/输出。



In [18]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
llm = OpenAI(temperature=.7)
template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title", 'era'], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis")


In [19]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=.7)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")


In [20]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)


In [21]:
overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})




> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'synopsis': "\n\nTragedy at Sunset on the Beach is set in Victorian England. It tells the story of a young couple, John and Sarah, who have been deeply in love since childhood. But their dreams of a life together are shattered when John is sent away to fight in the Crimean War.\n\nWhen John returns home, he is a changed man. He is no longer the same carefree young man that Sarah remembers. His experiences in war have left him broken and haunted, and Sarah struggles to reach him.\n\nMeanwhile, Sarah's father has arranged for her to marry a wealthy suitor, despite her protests. When she refuses to marry the man her father has chosen, her father threatens to disown her.\n\nThe play culminates in a fateful night at the beach. John and Sarah meet one last time to say goodbye and try to figure out a way to be together. But their love is not strong enough to withstand the obstacles in their way, and the tragedy at sunse

有时您可能想传递一些上下文以在链的每个步骤或链的后面部分中使用，但维护和链接输入/输出变量可能会很快变得混乱。使用 SimpleMemory 是一种方便的方式来管理这个问题并清理您的链。

举个例子，假设你使用之前介绍过的playwright SequentialChain，你想要在剧本中添加一些关于日期、时间和地点的上下文信息，并且使用生成的简介和评论来创建一些社交媒体发布文本。你可以将这些新的上下文变量添加为input_variables，或者我们可以添加一个SimpleMemory到链中来管理这个上下文：

In [22]:
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

llm = OpenAI(temperature=.7)
template = """You are a social media manager for a theater company. Given the title of play, the era it is set in, the date,time and location, the synopsis of the play, and the review of the play, it is your job to write a social media post for that play.

Here is some context about the time and location of the play:
Date and Time: {time}
Location: {location}

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
{review}

Social Media Post:
"""
prompt_template = PromptTemplate(input_variables=["synopsis", "review", "time", "location"], template=template)
social_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="social_post_text")

overall_chain = SequentialChain(
    memory=SimpleMemory(memories={"time": "December 25th, 8pm PST", "location": "Theater in the Park"}),
    chains=[synopsis_chain, review_chain, social_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["social_post_text"],
    verbose=True)

overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})




> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'time': 'December 25th, 8pm PST',
 'location': 'Theater in the Park',
 'social_post_text': "\nExperience the power and beauty of Victorian England in the heartbreaking love story, Tragedy at Sunset on the Beach! Join us this December 25th, 8pm PST at Theater in the Park. Don't miss this emotionally powerful play that examines complex issues such as class inequality and the power dynamics between men and women. Get your tickets now! #TragedyAtSunset #VictorianEngland #LoveStory #TheaterInThePark"}

序列化存储，将整个chain保存到本地，需要的时候再加载

In [23]:
from langchain import PromptTemplate, OpenAI, LLMChain
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=OpenAI(temperature=0), verbose=True)


In [24]:
llm_chain.save("llm_chain.json")


In [25]:
from langchain.chains import load_chain
chain = load_chain("llm_chain.json")


In [26]:
chain.run("whats 2 + 2")




> Entering new LLMChain chain...
Prompt after formatting:
Question: whats 2 + 2
 
Answer: Let's think step by step.

> Finished chain.


' 2 + 2 = 4'

单独保存一部分到本地

In [27]:
llm_chain.prompt.save("prompt.json")


In [29]:
llm_chain.llm.save("llm.json")


In [30]:
config = {
    "memory": None,
    "verbose": True,
    "prompt_path": "prompt.json",
    "llm_path": "llm.json",
    "output_key": "text",
    "_type": "llm_chain"
}
import json
with open("llm_chain_separate.json", "w") as f:
    json.dump(config, f, indent=2)

In [31]:
chain = load_chain("llm_chain_separate.json")
chain.run("whats 2 + 2")




> Entering new LLMChain chain...
Prompt after formatting:
Question: whats 2 + 2
 
Answer: Let's think step by step.

> Finished chain.


' 2 + 2 = 4'